In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [2]:
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout
from tensorflow.keras.applications import MobileNetV2, densenet , resnet50, VGG16


In [3]:
# Constants
DATA_DIR = '../input/dog-breed-identification/'
TRAIN_DIR = DATA_DIR + 'train/'
TEST_DIR = DATA_DIR + 'test/'
BATCH_SIZE = 32
INPUT_SIZE = 224
NUM_CLASSES = 120
SEED = 42

In [4]:
! ls $DATA_DIR

labels.csv  sample_submission.csv  test  train


In [5]:
import pandas as pd
train = pd.read_csv(DATA_DIR + 'labels.csv')
train.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [6]:
def append_ext(fn):
    return fn+".jpg"
train["id"]=train["id"].apply(append_ext)

In [7]:
breeds_20 = train.breed.value_counts().head(NUM_CLASSES).index
train = train[train.breed.isin(breeds_20)]

In [8]:
breeds_20

Index(['scottish_deerhound', 'maltese_dog', 'afghan_hound', 'entlebucher',
       'bernese_mountain_dog', 'shih-tzu', 'pomeranian', 'great_pyrenees',
       'basenji', 'samoyed',
       ...
       'german_shepherd', 'tibetan_mastiff', 'giant_schnauzer', 'walker_hound',
       'otterhound', 'komondor', 'golden_retriever', 'brabancon_griffon',
       'eskimo_dog', 'briard'],
      dtype='object', length=120)

In [9]:
train

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07.jpg,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97.jpg,dingo
2,001cdf01b096e06d78e9e5112d419397.jpg,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d.jpg,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62.jpg,golden_retriever
...,...,...
10217,ffd25009d635cfd16e793503ac5edef0.jpg,borzoi
10218,ffd3f636f7f379c51ba3648a9ff8254f.jpg,dandie_dinmont
10219,ffe2ca6c940cddfee68fa3cc6c63213f.jpg,airedale
10220,ffe5f6d8e2bff356e9482a80a6e29aac.jpg,miniature_pinscher


In [10]:
submission = pd.read_csv(DATA_DIR + 'sample_submission.csv')
submission.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
1,00102ee9d8eb90812350685311fe5890,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
2,0012a730dfa437f5f3613fb75efcd4ce,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
3,001510bc8570bbeee98c8d80c8a95ec1,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
4,001a5f3114548acdefa3d4da05474c2e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333


In [11]:
breed_labels = list(breeds_20.values)
# print(len(breed_labels))
train.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07.jpg,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97.jpg,dingo
2,001cdf01b096e06d78e9e5112d419397.jpg,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d.jpg,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62.jpg,golden_retriever


In [12]:
def create_model(lr=0.0001, dropout=None):
    
    
    #base = InceptionV3(weights='imagenet', include_top=False, input_shape=(INPUT_SIZE, INPUT_SIZE, 3))
    #base = Xception(weights='imagenet', include_top=False, input_shape=(INPUT_SIZE, INPUT_SIZE, 3))
    #base = VGG16(weights='imagenet', include_top=False, input_shape=(INPUT_SIZE, INPUT_SIZE, 3))
    #base = ResNet50(weights='imagenet', include_top=False, input_shape=(INPUT_SIZE, INPUT_SIZE, 3))
    base = MobileNetV2(weights='imagenet', include_top=False, input_shape=(INPUT_SIZE, INPUT_SIZE, 3))
    #base.layers.pop()   # Remove the last layer (softmax classifier)

    for layer in base.layers:
        layer.trainable = False
    headmodel = base.output
#     headmodel = tf.keras.layers.GlobalAveragePooling2D()(headmodel)
#     model.add(GlobalAveragePooling2D())
#     model.add(Dense(NUM_CLASSES, activation='softmax'))
    headmodel = Flatten()
    headmodel = tf.keras.layers.Dense(NUM_CLASSES, activation= 'softmax')(headmodel)

#     adam = Adam(lr=lr)
#     sgd = SGD(lr=0.1, momentum=0.95, nesterov=False)
    model = tf.keras.Model(inputs=base.input, outputs= headmodel)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr) ,
                  loss = tf.keras.losses.categorical_crossentropy, metrics=['accuracy'],  )

    return model

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
train_set, val_set = train_test_split(train, test_size=0.20, stratify=train['breed'], random_state=SEED)

In [14]:
train_set.head()

,id,breed
1926,2f812a0cb6977bbad1a167e1ef4ae2ac.jpg,gordon_setter
735,123c19c8d168e7704273cb7174351821.jpg,vizsla
2304,396ef6a16b8640e1e5b69f872540776f.jpg,australian_terrier
3838,5f14fac852ee51524997243f086e4ea2.jpg,norwich_terrier
5252,8463aa43d88bee057082434ccc806bb0.jpg,bernese_mountain_dog


In [22]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255.,
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [23]:
train_generator = train_datagen.flow_from_dataframe(dataframe=train_set,
                                                directory=TRAIN_DIR,
                                                x_col='id',
                                                y_col='breed',
                                                class_mode='categorical',
                                                classes=breed_labels,
                                                has_ext=False,
                                                batch_size=BATCH_SIZE,
                                                shuffle=True,
                                                seed=SEED,
                                                target_size=(INPUT_SIZE, INPUT_SIZE)
                                               )

Found 8177 validated image filenames belonging to 120 classes.


In [24]:
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255.
)

In [25]:
val_generator = val_datagen.flow_from_dataframe(dataframe=val_set,
                                                directory=TRAIN_DIR,
                                                x_col='id',
                                                y_col='breed',
                                                class_mode='categorical',
                                                classes=breed_labels,
                                                has_ext=False,
                                                batch_size=BATCH_SIZE,
                                                shuffle=True,
                                                seed=SEED,
                                                target_size=(INPUT_SIZE, INPUT_SIZE)
                                               )

Found 2045 validated image filenames belonging to 120 classes.


In [26]:
model = create_model()

In [27]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
____________________________________________________________________________________________

In [ ]:
EPOCHS = 50
model.fit(train_generator, epochs=EPOCHS, validation_data=val_generator,validation_freq=1)

Epoch 1/50
256/256 [==============================] - ETA: 0s - loss: 4.2441 - accuracy: 0.0987